# Configure a Scenario for FloodAdapt

In this notebook we are going to setup a scenario for FloodAdapt to run. This includes a weather event, any adaptation measures and future projections.


## **Step 0:** Import dependencies

In [ ]:
from pathlib import Path
from pprint import pprint
import tomli_w
from DT_flood.utils.workflow_utils import create_workflow_config, run_fa_scenario_workflow

## **Step 1:** Set paths to folders

In [ ]:
# Path to the FloodAdapt database for which we are configuring scenarios
fa_database = Path("/home/wotromp/DE372/scheveningen")

# Path to data folder. If (relative) file paths are used when specifying data sets, it will be looked for in this folder.
# data_folder = Path("/home/wotromp/InterTwin/Data")

# OPTIONAL: HydroMT data catalogue from which to fetch data. The order matters here, as datasets are looked for in a catalogue in order from first catalogue name in list to last.
data_catalogues = []

## **Step 2:** Configure the weather Event

Here we will specify everything needed to setup an event. For now we will only consider scenarios which involve just a single event.

To fully specify an event we will need the following:
- A name for referencing
- Start and end time and dates
- Forcing data for the hazard models. Either paths to data files or keys for data catalogues. See HydroMT (SFINCS,WFlow) for naming conventions of data variables
    - When using a single dataset for meteorological data this needs to contain precipitation, wind speeds, and pressure. Otherwise use individual data sources for each

WFlow requires two sets of forcings, one for the warmup run, and one for the event run. See HydroMT WFlow `setup_precip_forcing` and `setup_temp_pet_forcing` which data variables should be included (conveniently ERA5 provides them all)

In [ ]:
# Name of event
event_name = 'DT-climate_feb39'

# Start and end times of the event
start_time = '2039-02-13 00:00:00'
end_time = '2039-02-17 00:00:00'

# Forcing data for the sfincs model.
sfincs_meteo = ''
sfincs_waterlevel = ''

sfincs_forcing_offshore = "/home/wotromp/DE372/data/dt-climate_event_feb2039.nc"


In [ ]:
event_dict = {
    'name': event_name,
    'start_time': start_time,
    'end_time': end_time,
    "data_catalogues":  [str(path) for path in data_catalogues],
    'sfincs_forcing': {
        'meteo': sfincs_meteo,
        'waterlevel': sfincs_waterlevel
    },
    "sfincs_forcing_offshore": sfincs_forcing_offshore
}

## **Step 3:** Configure projections

Here we specify, where applicable, the projected future changes relevant to our scenario. These come in two flavors: physical projections and socio-economic changes. The former contains changes on the hazard side (sea level rise, land subsidence etc.) while the latter contains changes to the impact side (population growth, economic growth etc.).\
The supported projections are:\
\
**Physical Projections:**
- Sea level rise
- Land subsidence
- Rainfall increase
- Storm frequency increase

**Socio-Economic Projections:**
- Population growth
- Economic growth
- New developments:
    - Newly developed area
    - Elevation of newly developed area
    - Population growth in newly developed area

In [ ]:
# Name of projection
projection_name = 'empty_projection'

# Physical projections
sea_level_rise = 0
rainfall_increase = 0


# Socio-Economic projections
pop_growth = 0
economic_growth = 0

In [ ]:
proj_dict = {
    'name': projection_name,
    'physical_projection': {
        'sea_level_rise': sea_level_rise,
        'rainfall_increase': rainfall_increase
    },
    'socio_economic_change': {
        'population_growth_existing': pop_growth,
        'economic_growth': economic_growth
    }
}

## **Step 4:** Configure Measures

Here we configure all the measure we want to implement for this scenario. These come in either Impact or Hazard types, either reducing the vulnerability to a hazard, or dampen the hazard itself. In either case we need to specify in which area the measure acts by specifying a geojson file or in the case of Impact types an area name recognized by the aggregation area file. For the Impact type we also need to specify which building type to act on.\
The supported measures are:\
\
**Hazard Type:**
- Floodwall
- Pump
- Water square
- Green infrastructure
- Water storage

**Impact Type:**
- Elevate properties
- Floodproof properties
- Buyout properties

All the measures needed for the scenario are collected into a single strategy.

In [ ]:
# Run this cell to reset the measures to include in the scenario
strategy_name = 'empty_strategy'
strategy = {'name': strategy_name}

In [ ]:
# Rerun this cell multiple times to include multiple measures, everytime with updated variables appropriate to the to-be-added measure

# Name of measure
measure_name = 'test_measure1'

# Measure type
measure_type = 'floodwall'
measure_area = str(fa_database / 'HumberDelta_weir.geojson')

# These settings depend on the measure, already in a FloodAdapt format
measure_misc = {"elevation": {"value": 2, "units": "meters"}}

measure_dict = {
    'name': measure_name,
    'type': measure_type,
    'selection': measure_area,
    'misc': measure_misc,
}

In [ ]:
strategy.update({f"measure{len(strategy)}": measure_dict})

## **Step 5:** Collect everything into a single Scenario

Now that we have configured each individual element of a scenario (event, projection, measure) we can collect everything together into a single scenario definition

In [ ]:
scenario = {
    'name': "DT-climate_baseline",
    'event': event_dict,
    'projection': proj_dict,
    'strategy': strategy,
}

In [ ]:
pprint(scenario)

In [ ]:
# Write to toml file. The _toplevel suffix indicated that this is a toml file also configuring non-FloodAdapt functionalities
scenario_fn = fa_database / f"{scenario['name']}_toplevel.toml"

with open(scenario_fn, 'wb+') as f:
    tomli_w.dump(scenario, f)

## **Step 6:** Launch workflow

With a complete definition of the scenario we can run the workflow. First we export the cwl config file based on the scenario settings we just created

In [1]:
from pathlib import Path
from pprint import pprint
import tomli_w
from DT_flood.utils.workflow_utils import create_workflow_config, run_fa_scenario_workflow

In [2]:
database_fn = Path("/home/wotromp/DE372/scheveningen")
scenario_name = 'DT-climate-2039proj'

In [3]:
create_workflow_config(database_fn, scenario_name)

Saving cwl config to /home/wotromp/DE372/scheveningen/cwl_config_DT-climate-2039proj.yml
Workflow file: /home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl


INFO /home/wotromp/miniforge3/envs/de372_demo/bin/cwltool 3.1.20241112140730
INFO Resolved '/home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl' to 'file:///home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl'


Write Config file /home/wotromp/DE372/scheveningen/cwl_config_DT-climate-2039proj.yml to folder /home/wotromp/DE372/scheveningen/cwl_config_DT-climate-2039proj.yml


In [5]:
run_fa_scenario_workflow(database_fn, scenario_name)

Validating workflow
Running cwltool --validate "/home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl" "/home/wotromp/DE372/scheveningen/cwl_config_DT-climate-2039proj.yml"


INFO /home/wotromp/miniforge3/envs/de372_demo/bin/cwltool 3.1.20241112140730
INFO Resolved '/home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl' to 'file:///home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl'


../DT_flood/workflows/run_fa_scenario.cwl is valid CWL. No errors detected in the inputs.
Executing workflow
Running cwltool --outdir "/home/wotromp/DE372" --preserve-entire-environment "/home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl" "/home/wotromp/DE372/scheveningen/cwl_config_DT-climate-2039proj.yml"


INFO /home/wotromp/miniforge3/envs/de372_demo/bin/cwltool 3.1.20241112140730
INFO Resolved '/home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl' to 'file:///home/wotromp/Repos/DT-flood/DT_flood/workflows/run_fa_scenario.cwl'
INFO [workflow ] start
INFO [workflow ] starting step init_scenario
INFO [step init_scenario] start
INFO [job init_scenario] /tmp/ddw34f2v$ python \
    /tmp/ddw34f2v/init_fa_database.py \
    /tmp/ddw34f2v/scheveningen \
    DT-climate-2039proj
Make extra copies of input folders
Model dir already exists and files might be overwritten: /tmp/ddw34f2v/scheveningen/static/templates/overland/gis.
Creating output folder at /tmp/ddw34f2v/scheveningen/output/scenarios/DT-climate-2039proj
INFO [job init_scenario] Max memory used: 525MiB
INFO [job init_scenario] completed success
INFO [step init_scenario] completed success
INFO [workflow ] starting step run_sfincs_offshore
INFO [step run_sfincs_offshore] start
INFO [workflow run_sfincs_offshore] start
INFO [

{
    "fa_database_out": {
        "location": "file:///home/wotromp/DE372/scheveningen",
        "basename": "scheveningen",
        "class": "Directory",
        "listing": [
            {
                "class": "File",
                "location": "file:///home/wotromp/DE372/scheveningen/cwl_config_DT-climate_feb39_floodwall.yml",
                "basename": "cwl_config_DT-climate_feb39_floodwall.yml",
                "size": 962,
                "checksum": "sha1$afc8b58ea16fd713a0cf610683c896b469bd205e",
                "path": "/home/wotromp/DE372/scheveningen/cwl_config_DT-climate_feb39_floodwall.yml"
            },
            {
                "class": "File",
                "location": "file:///home/wotromp/DE372/scheveningen/test_scenario_offshore_toplevel.toml",
                "basename": "test_scenario_offshore_toplevel.toml",
                "size": 528,
                "checksum": "sha1$3ca852c32c3aa26fcb2f45d9bc655472c452488d",
                "path": "/home/wotromp/

INFO Final process status is success
